In [1]:
#Create GPS data with desired accuracy
using LinearAlgebra
using ForwardDiff
using SatelliteDynamics
using DelimitedFiles

In [8]:
#accurate dynamics

#epc is the current epoch (current time), and x is the state

#accurate dynamics. considers gravity, J2, drag, solar radiation
#pressure, and the effect of other bodies (sun/moon)

#ẋ = f(x,t)
function ground_truth_dynamics(x, epc)
    
    r = x[1:3] #satellite position in inertial frame
    v = x[4:6] #satellite velocity in inertial frame
        
    #gives a rotation matrix
    PN = bias_precession_nutation(epc)
    
    #Compute the sun and moon positions in ECI frame
    r_sun = sun_position(epc)
    r_moon = moon_position(epc)
    
    #define the acceleration variable
    a = zeros(eltype(x), 3)
    
    #compute acceleration caused by Earth gravity (includes J2)
    #modeled by a spherical harmonic gravity field
    #look up this term. seems to give a rotation matrix
    PN = bias_precession_nutation(epc)
    Earth_r = earth_rotation(epc)
    rpm  = polar_motion(epc) 

    R = rpm*Earth_r*PN
    #10th order gravity
    n_grav = 10
    m_grav = 10
    #main contribution in acceleration (seemed to not be equal to the Series Expansion of gravity)
    a+= accel_gravity(x, R, n_grav, m_grav)
    
    a_grav = accel_gravity(x, R, n_grav, m_grav)
    
    #this is the gravity code that is used to find the difference between higher order model and J2 model 
    ###########################################################################################################
    #compute the gravitational acceleration based off the series expansion up to J2
    μ = 3.986004418e14 #m3/s2
    J2 = 1.08264e-3 
        
    a_2bp = (-μ*r)/(norm(r))^3
    
    Iz = [0,0,1]
    
    a_J2 = ((3*μ*J2*R_EARTH^2)/(2*norm(r)^5))*((((5*dot(r, Iz)^2)/norm(r)^2)-1)*r - 2*dot(r,Iz)*Iz)     

    a_process = a_2bp + a_J2
    
    #this is actual higher order
    higher_order_unmodeled = a_grav - a_process
    ############################################################################################################
    
    
    
    
    #atmospheric drag using true of date coordinate system
    ############################################################################################################
    #atmospheric drag
    #compute the atmospheric density from density harris priester model
     ρ_est = density_harris_priester(epc,r)

     #ρ = 1.15e-12 #fixed atmospheric density in kg/m3
    
     k_ρ_true = 1.0
     #sinusoidal
     #k_ρ_true = 0.1*sin(0.001*(epc-epc0))+1
    
     ρ_true = ρ_est*k_ρ_true
    
     #computes acceleration due to drag in inertial directions
     cd = 2.0 #drag coefficient
     area_drag = 0.1 #in m2 #area normal to the velocity direction
     m = 1.0 #assuming unit mass
    
     #drag from satellite dynamics.jl
     a += accel_drag(x, ρ_true, m, area_drag, cd, Array{Real,2}(PN))
###############################################################################################################
    
    
    #drag in ECI coordinates
    
    ##########################################3
    
    #k_ρ = 1.1
    
    #drag equation
    #c_d = 2.0 #drag coefficient (dimensionless)
    
    #A = 0.1  #cross sectional area
    
    #rotation of the earth (rad/s)
    #ω_earth = [0,0, OMEGA_EARTH]
    
    #v_rel = v - cross(ω_earth, r)
    
    #set a constant atmospheric density
    #ρ_est = density_harris_priester(epc,r)
    
    #f_drag = -0.5*c_d*(A)*k_ρ*ρ_est*norm(v_rel)*v_rel
    
    #a+= f_drag
    ##########################################
    
    #Solar Radiation Pressure
    area_srp = 1.0
    coef_srp = 1.8
    a += accel_srp(x, r_sun, m, area_srp, coef_srp)
    a_srp = accel_srp(x, r_sun, m, area_srp, coef_srp)
    
    #acceleration due to external bodies
    a+= accel_thirdbody_sun(x, r_sun)
    a_sun = accel_thirdbody_sun(x, r_sun)
    
    a+= accel_thirdbody_moon(x, r_moon)
    a_moon = accel_thirdbody_moon(x, r_moon)
    
    #commented out higher order gravities
    a_unmodeled = a_srp + a_sun + a_moon + higher_order_unmodeled
            
    xdot = x[4:6]
    vdot = a
    
    x_dot = [xdot; vdot]
    
    return x_dot, a_unmodeled, higher_order_unmodeled, ρ_est 
    
end

ground_truth_dynamics (generic function with 1 method)

In [3]:
epc0 = Epoch(2012, 11, 8, 12, 0, 0, 0.0)

Epoch(2012-11-08T11:59:25.000Z)

In [4]:
#wildlife tracking orbit
iss1 = [6871e3, 0.0004879, 90.6391, 194.5859, 151.2014, 190]; 

# Convert osculating elements to Cartesean state
# returns position and velocity (m, m/s). This is the intial position
eci0_1 = sOSCtoCART(iss1, use_degrees=true)

#find the period of the orbit (seconds). only dependent on semi major axis
T = orbit_period(iss1[1])

#final time of simulation
epcf = epc0 + T

Epoch(2012-11-08T13:33:53.144Z)

In [5]:
#testing hand-tuned Q
pose_std_dynamics = 3e-5 #these pose and velocity values are pretty good
velocity_std_dynamics = 0.5e-5

Q = I(6).*[ones(3)*(pose_std_dynamics^2)/3; ones(3)*(velocity_std_dynamics^2)/3]

6×6 Matrix{Float64}:
 3.0e-10  0.0      0.0      0.0          0.0          0.0
 0.0      3.0e-10  0.0      0.0          0.0          0.0
 0.0      0.0      3.0e-10  0.0          0.0          0.0
 0.0      0.0      0.0      8.33333e-12  0.0          0.0
 0.0      0.0      0.0      0.0          8.33333e-12  0.0
 0.0      0.0      0.0      0.0          0.0          8.33333e-12

In [6]:
function satRK4(x, tc)
    
    f1,_,_, _ = ground_truth_dynamics(x, tc) 
    f2,_,_, _ = ground_truth_dynamics(x+0.5*h*f1, tc+h/2)
    f3,_,_, _ = ground_truth_dynamics(x+0.5*h*f2, tc+h/2)
    f4,_,_, _ = ground_truth_dynamics(x+h*f3, tc+h)
    
    xnext = x+(h/6.0)*(f1+2*f2+2*f3+f4)
            
    return xnext
    
end

satRK4 (generic function with 1 method)

In [10]:
#timestep
h = 25

#initial state
x_0 = eci0_1

#number of orbits to simulate
orbit_num = 5

#final time
Tf = T*orbit_num

#run the rk4
t = Array(range(0,Tf, step=h)) #create a range to final time at constant time step
    
all_x = zeros(length(x_0), length(t)) #variable to store all x
#all_fdrag = zeros(3, length(t))

all_x[:,1] = x_0 #set the initial state


for k=1:(length(t) - 1)

    #get the current time
    current_t = epc0+t[k]
    
    all_x[:,k+1] = satRK4(all_x[:,k], current_t) #calculate the next state

end

#contains all the ground truth states
x_hist = all_x

6×1134 Matrix{Float64}:
    -6.29129e6     -6.34874e6     -6.40132e6  …     -6.20686e6     -6.27069e6
    -1.66265e6     -1.67551e6     -1.68709e6        -1.64676e6     -1.66135e6
    -2.21616e6     -2.03517e6     -1.85262e6        -2.4534e6      -2.27471e6
 -2394.97       -2200.95       -2005.21          -2648.63       -2457.13
  -540.153       -488.913       -437.291          -608.782       -558.01
  7206.15        7271.59        7331.44       …   7110.95        7183.72

In [12]:
# save the ground truth data

writedlm("groundtruth_data_chief.txt", x_hist)

In [22]:
#timestep
h = 25

#initial state
x_0 = eci0_1
x_0 = x_0 + diagm([100,100,100, 10,10,10])*randn(6)

#number of orbits to simulate
orbit_num = 5

#final time
Tf = T*orbit_num

#run the rk4
t = Array(range(0,Tf, step=h)) #create a range to final time at constant time step
    
all_x = zeros(length(x_0), length(t)) #variable to store all x
#all_fdrag = zeros(3, length(t))

all_x[:,1] = x_0 #set the initial state


for k=1:(length(t) - 1)

    #get the current time
    current_t = epc0+t[k]
    
    all_x[:,k+1] = satRK4(all_x[:,k], current_t) #calculate the next state

end

#contains all the ground truth states
x_hist1 = all_x

6×1134 Matrix{Float64}:
    -6.29127e6     -6.34855e6     -6.40097e6  …     -6.02679e6     -6.10227e6
    -1.66269e6     -1.67548e6     -1.68697e6        -1.6051e6      -1.62274e6
    -2.21626e6     -2.03507e6     -1.85231e6        -2.89293e6     -2.71901e6
 -2388.38       -2194.35       -1998.59          -3112.19       -2926.16
  -536.87        -485.627       -434.0            -730.491       -680.984
  7214.47        7279.92        7339.77       …   6913.43        6999.74

In [23]:
writedlm("groundtruth_data_dep3.txt", x_hist1)